In [1]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from pprint import pprint

df_jan = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')
# df = pd.concat([df_jan, df_feb], axis=0, ignore_index=True)
df_jan.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3066766 entries, 0 to 3066765
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[us]
 2   tpep_dropoff_datetime  datetime64[us]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee           

In [ ]:
#### QUESTION 2 ####

In [3]:
res = df_jan.tpep_dropoff_datetime - df_jan.tpep_pickup_datetime
# print(res.info())
# df_jan[res.isna()].sample() #[['tpep_pickup_datetime', 'tpep_dropoff_datetime']].sample()
(res.dt.total_seconds()/60).std()

np.float64(42.59435124195458)

In [4]:
#### QUESTION 3 ####

In [5]:
df3 = df_jan.copy()
df3['duration_min'] = (df3.tpep_dropoff_datetime - df3.tpep_pickup_datetime).dt.total_seconds()/60
df3 = df3[df3['duration_min'].notna()]
df3 = df3.loc[(df3['duration_min'] >= 1) & (df3['duration_min'] <= 60)]

print( (len(df3) / len(df_jan)))

0.9812202822125979


In [6]:
#### QUESTION 4 ####

In [7]:
categorical = ['PULocationID', 'DOLocationID']
dict4 = df3[categorical].astype(str).to_dict('records')
# pprint(dict4[0:5])

dv = DictVectorizer(sparse=True)
df4 = dv.fit_transform(dict4)
# pprint(x)
# print(x.shape)
len(dv.feature_names_)

515

In [8]:
#### QUESTION 5 ####

In [9]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error

# type(df4)
labels = df3['duration_min']
reg = LinearRegression().fit(df4, labels)
predict = reg.predict(df4)
root_mean_squared_error(labels, predict)

7.649261932073194

In [10]:
#### QUESTION 6 ####

In [11]:
df_feb = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

df_feb['duration_min'] = (df_feb.tpep_dropoff_datetime - df_feb.tpep_pickup_datetime).dt.total_seconds()/60
df_feb = df_feb[df_feb['duration_min'].notna()]
df_feb = df_feb.loc[(df_feb['duration_min'] >= 1) & (df_feb['duration_min'] <= 60)]

dict6 = df_feb[categorical].astype(str).to_dict('records')
input6 = dv.transform(dict6)

labels6 = df_feb['duration_min']

In [12]:
predict6 = reg.predict(input6)
root_mean_squared_error(labels6, predict6)

7.811818966384802

In [ ]:
df_feb.info()

In [1]:
#### rough ####

In [2]:
from prefect import flow, get_run_logger, task
from prefect.cache_policies import INPUTS
from prefect.docker import DockerImage
import pandas as pd
from sklearn.feature_extraction import DictVectorizer

In [3]:
def data_loader(url: str = 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-03.parquet'):
    df = pd.read_parquet(url)
    return df

In [4]:
df = data_loader()

In [18]:
dv = DictVectorizer()
    
df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
df.duration = df.duration.dt.total_seconds() / 60
df = df[(df.duration >= 1) & (df.duration <= 60)]


categorical = ['PULocationID', 'DOLocationID']
df[categorical] = df[categorical].astype(str)
dicts = df[categorical].to_dict(orient='records')
X = dv.fit_transform(dicts)
y = df['duration']

In [19]:
type(X)
X.shape

(3316216, 518)

In [20]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X, y)

LinearRegression()

In [21]:
model.intercept_

np.float64(24.77860328111726)